In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
%matplotlib inline
#import seaborn as sns
from sklearn.metrics import roc_auc_score
import optuna
from optuna.samplers import TPESampler
from sklearn.preprocessing import StandardScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/advanced-dls-spring-2021/submission.csv
/kaggle/input/advanced-dls-spring-2021/train.csv
/kaggle/input/advanced-dls-spring-2021/test.csv


In [12]:
train = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
test = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/test.csv")
data = train
data.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0


In [16]:
# Для вашего удобства списки с именами разных колонок

# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'




In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[feature_cols], train[target_col], test_size = 0.2, random_state = 43)

**XGBOOST**

In [116]:
import xgboost as xgb
std = StandardScaler()

X = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
X_test = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/test.csv")


X['TotalSpent'] = pd.to_numeric(X['TotalSpent'], errors='coerce').fillna(0)
X.drop_duplicates(inplace=True) 

X_test['TotalSpent'] = pd.to_numeric(X_test['TotalSpent'], errors='coerce').fillna(0)



In [117]:
y = X[target_col]
X = X[feature_cols]

X = pd.get_dummies(X, columns=cat_cols);
X_test = pd.get_dummies(X_test, columns=cat_cols)

X[num_cols] = std.fit_transform(X[num_cols])
X_test[num_cols] = std.transform(X_test[num_cols])

Xg_train, Xg_valid, yg_train, yg_valid = train_test_split(X, y, train_size=0.8, random_state=69)


In [120]:
def objective(trial):
    param = {
        'tree_method':'gpu_hist',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 10000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBClassifier(**param)  
    
    model.fit(Xg_train,yg_train,eval_set=[(Xg_valid,yg_valid)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(Xg_valid)
    
    err = roc_auc_score(yg_valid, preds)
    
    return err

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = TPESampler(seed=24)
study = optuna.create_study(study_name="xgboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20, show_progress_bar = True)
print('Best trial:', study.best_trial.params)

In [122]:
trial = study.best_trial
xgboost = xgb.XGBClassifier(**trial.params)
xgboost.fit(Xg_train, yg_train)

XGBClassifier(alpha=2.492059409816763, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.008204132440644262,
              learning_rate=0.018, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=53, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [90]:
Xg_train

,ClientPeriod,MonthlySpending,TotalSpent,Churn,Sex_Female,Sex_Male,IsSeniorCitizen_0,IsSeniorCitizen_1,HasPartner_No,HasPartner_Yes,...,HasMovieSubscription_Yes,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
4668,1.448175,-1.488991,-0.399554,0,1,0,1,0,1,0,...,0,0,0,1,1,0,1,0,0,0
941,-0.794241,-0.358847,-0.701837,0,1,0,1,0,0,1,...,0,0,1,0,1,0,1,0,0,0
2055,-0.468072,1.311511,-0.023820,0,1,0,1,0,1,0,...,1,0,1,0,0,1,0,0,0,1
396,1.570489,-1.333223,-0.201219,0,1,0,1,0,0,1,...,0,0,0,1,1,0,1,0,0,0
3770,1.162777,0.531016,1.182383,0,0,1,1,0,0,1,...,1,0,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,-0.794241,1.056317,-0.466260,0,1,0,0,1,1,0,...,1,1,0,0,0,1,0,0,1,0
439,0.755065,-0.159995,0.345346,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,0,0
1628,-0.264216,-0.338962,-0.390739,0,1,0,1,0,1,0,...,0,1,0,0,0,1,0,1,0,0
2673,-0.753470,-1.492305,-0.892525,0,0,1,1,0,1,0,...,0,1,0,0,1,0,0,0,0,1


In [125]:
preds_xgboost = xgboost.predict_proba(X_test)[:, 1]
X_sub = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/test.csv')

submission = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/submission.csv')

#submission.drop(submission.columns[0], axis = 1)


submission['Churn'] = preds_xgboost
submission.to_csv('xgboost_submission.csv', index = False)


**Catboost**




In [38]:
from catboost import CatBoostClassifier

In [48]:
train = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
test = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/test.csv")
data = train
X_train, X_test, y_train, y_test = train_test_split(train[feature_cols], train[target_col], test_size = 0.2, random_state = 43)

In [49]:
def objective(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 12),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type="Bayesian",
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type = "IncToDec",
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False,
        task_type="GPU",
        devices='0:1',
        cat_features = cat_cols
        
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return roc_auc_score(y_test, y_pred)

In [51]:
from sklearn.metrics import roc_auc_score
import optuna
from optuna.samplers import TPESampler


In [52]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20, show_progress_bar = True)

  0%|          | 0/20 [00:00<?, ?it/s]

In [55]:
#cat.fit(X_train,y_train,verbose=False, plot=True, cat_features = cat_cols)
trial = study.best_trial
model = CatBoostClassifier(**trial.params, verbose=False,task_type="GPU",
        devices='0:1',
        bootstrap_type="Bayesian",
        od_type="IncToDec",
        cat_features = cat_cols)
trial.params
#y_pred = model.predict(X_test)

{'iterations': 640,
 'learning_rate': 0.046939005475074985,
 'depth': 11,
 'l2_leaf_reg': 41.026179099979494,
 'random_strength': 1.2137101089229371e-08,
 'bagging_temperature': 0.400701147486561,
 'od_wait': 19}

In [57]:
train = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
y = train[target_col]
x_fit = train[feature_cols]
params = trial.params

In [ ]:
#model.fit(x_fit, y)

In [58]:
model1 = CatBoostClassifier(**params, verbose=False,task_type="GPU",
        devices='0',
        bootstrap_type="Bayesian",
        od_type = "IncToDec",
        cat_features = cat_cols)
model1.fit(x_fit, y)



In [60]:
from sklearn.pipeline import make_pipeline

In [70]:
X_train += pd.get_dummies(X_train, columns = cat_cols)


Попробуем просто взять предсказания обеих моделей, добавить их к численным колонкам и обучить катбуст на всем этом


In [129]:
catboost_preds = model1.predict_proba(X_sub)[:, 1]
preds_xgboost

array([0.14978038, 0.4792323 , 0.6342453 , ..., 0.6813063 , 0.48961094,
       0.11876315], dtype=float32)

In [176]:
train = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
test = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/test.csv")
data = train
X_train, X_test, y_train, y_test = train_test_split(train[feature_cols], train[target_col], test_size = 0.2, random_state = 43)
X = data[feature_cols]
y = data[target_col]


In [177]:
train_preds_catboost = model1.predict_proba(X_train)[:, 1]

X_xg = X_train.copy()
X_xg['TotalSpent'] = pd.to_numeric(X_xg['TotalSpent'], errors='coerce').fillna(0)
X_xg = pd.get_dummies(X_xg, columns=cat_cols);
X_xg[num_cols] = std.fit_transform(X_xg[num_cols])

train_preds_xg = xgboost.predict_proba(X_xg)[:, 1]

test_preds_catboost = model1.predict_proba(X_test)[:, 1]
Xt_xg = X_test.copy()
Xt_xg['TotalSpent'] = pd.to_numeric(Xt_xg['TotalSpent'], errors='coerce').fillna(0)
Xt_xg = pd.get_dummies(Xt_xg, columns=cat_cols);
Xt_xg[num_cols] = std.fit_transform(Xt_xg[num_cols])

test_preds_xg = xgboost.predict_proba(Xt_xg)[:, 1]




In [178]:
X_train["preds_catboost"] = train_preds_catboost
X_test["preds_catboost"] = test_preds_catboost


In [179]:
X_train["preds_xgboost"] = train_preds_xg
X_test["preds_xgboost"] = test_preds_xg

In [161]:
X_train

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,...,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,preds_catboost,preds_xgboost
1497,72,116.05,8404.9,Male,1,Yes,No,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),0.022983,0.142670
4668,68,20.05,1386.9,Female,0,No,No,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),0.004357,0.109306
4327,71,109.70,7904.25,Female,1,Yes,No,Yes,No,Fiber optic,...,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),0.016996,0.142670
2828,64,65.80,4068,Female,0,Yes,Yes,Yes,No,DSL,...,No,No,Yes,Yes,No,One year,No,Mailed check,0.015615,0.139139
2768,30,66.30,1923.5,Female,0,Yes,Yes,Yes,No,DSL,...,No,No,Yes,No,Yes,Month-to-month,No,Bank transfer (automatic),0.058099,0.203931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,44,74.85,3268.05,Female,1,No,No,Yes,No,DSL,...,No,Yes,Yes,Yes,Yes,One year,No,Bank transfer (automatic),0.146462,0.168422
2064,13,44.80,559.2,Male,0,Yes,Yes,Yes,No,DSL,...,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),0.176018,0.292029
2325,15,99.10,1426.4,Female,0,No,No,Yes,No,Fiber optic,...,Yes,No,No,Yes,Yes,Month-to-month,Yes,Credit card (automatic),0.596277,0.404952
2303,26,105.75,2710.25,Male,1,Yes,Yes,Yes,Yes,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,0.530521,0.462434


In [180]:
def objective(trial):
    params = {'od_type' : "IncToDec",
             'bootstrap_type':"Bayesian",
              'cat_features' : cat_cols,
             }
    model1 = CatBoostClassifier(
        **params,
        iterations=trial.suggest_int("iterations", 100, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 12),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        #random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        #bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False,
        task_type="GPU",
        devices='0:1',
    )
    model1.fit(X_train, y_train)
    y_pred = model1.predict(X_test)
    return roc_auc_score(y_test, y_pred)

In [181]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="catboost1", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=30, show_progress_bar = True)

  0%|          | 0/30 [00:00<?, ?it/s]

In [193]:
#cat.fit(X_train,y_train,verbose=False, plot=True, cat_features = cat_cols)
trial = study.best_trial
model = CatBoostClassifier(**trial.params, verbose=False,task_type="GPU",
        devices='0:1',
        bootstrap_type="Bayesian",
        od_type="IncToDec",
        cat_features = cat_cols)
trial.params
#y_pred = model.predict(X_test)

{'iterations': 327,
 'learning_rate': 0.09410765368954498,
 'depth': 7,
 'l2_leaf_reg': 27.851622084086536,
 'random_strength': 0.0002543262448784432,
 'bagging_temperature': 5.240459372584388,
 'od_wait': 35}

In [189]:
train = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
y = train[target_col]
x_fit = train[feature_cols]

train_preds_catboost = model1.predict_proba(x_fit)[:, 1]

X_xg = x_fit.copy()
X_xg['TotalSpent'] = pd.to_numeric(X_xg['TotalSpent'], errors='coerce').fillna(0)
X_xg = pd.get_dummies(X_xg, columns=cat_cols);
X_xg[num_cols] = std.fit_transform(X_xg[num_cols])
train_preds_xg = xgboost.predict_proba(X_xg)[:, 1]

params = trial.params
x_fit["preds_catboost"] = train_preds_catboost
x_fit["preds_xgboost"] = train_preds_xg

In [194]:
model.fit(x_fit, y)

In [195]:
X_sub = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/test.csv')
X_sub["preds_catboost"] = catboost_preds
X_sub["preds_xgboost"] = preds_xgboost


In [196]:

probs = model.predict_proba(X_sub)[:, 1]

submission = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/submission.csv')

#submission.drop(submission.columns[0], axis = 1)

submission['Churn'] = probs
submission.to_csv('stacking_submission.csv', index = False)


------------------

In [71]:
boost = make_pipeline(std, xgboost)

In [72]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

In [73]:
def objective(trial):
    model = StackingClassifier(
    estimators=[
        ("catboost", model1),
        ("xgboost", boost),
    ],
    final_estimator=CatBoostClassifier(
    logging_level='Silent',
    eval_metric="AUC:hints=skip_train~false",
    metric_period=1000,
    random_seed=43,
    grow_policy=trial.suggest_categorical('grow_policy', ["Lossguide", "Depthwise"]),
    l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
    learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
    max_depth=trial.suggest_int("depth", 4, 12),
    min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 10),
    n_estimators=trial.suggest_int("n_estimators", 10, 250),
    random_strength=11,
    subsample=0.1,
    ),
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return roc_auc_score(y_test, y_pred)

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="stacking", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20, show_progress_bar = True)

In [50]:
#cat.fit(X_train,y_train,verbose=False, plot=True, cat_features = cat_cols)
trial = study.best_trial
# model = CatBoostClassifier(**trial.params, verbose=False,task_type="GPU",
#         devices='0:1',
#         bootstrap_type="Bayesian",
#         od_type = "IncToDec",
#         cat_features = cat_cols)

trial.params

#y_pred = model.predict(X_test)

{'l2_leaf_reg': 28.272580044430782,
 'learning_rate': 0.013112992007873706,
 'depth': 4}

In [52]:
meta = CatBoostClassifier(
    **trial.params,
    logging_level='Silent',
    eval_metric="AUC:hints=skip_train~false",
    metric_period=1000,
    random_seed=2,
    min_data_in_leaf=10,
    n_estimators=10,
    random_strength=11,
    subsample=0.1,
)
stacking = StackingClassifier(
    estimators=[
        ("cat1", model1),
        ("cat2", model2),
    ],
    final_estimator=meta,
    n_jobs=1,
)

stacking.fit(x_fit, y)

StackingClassifier(estimators=[('cat1',
                                <catboost.core.CatBoostClassifier object at 0x7878a47aa530>),
                               ('cat2',
                                <catboost.core.CatBoostClassifier object at 0x7878a47aa6e0>)],
                   final_estimator=<catboost.core.CatBoostClassifier object at 0x7878a49b6650>,
                   n_jobs=1)

In [53]:
model = stacking

In [54]:
test_preds = model.predict_proba(X_test)
roc_auc_score(y_test, test_preds[:, 1])

0.8820778540118163

In [55]:
X_sub = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/test.csv')

probs = model.predict_proba(X_sub)[:, 1]

submission = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/submission.csv')

#submission.drop(submission.columns[0], axis = 1)
y = model.predict(X_sub)

submission['Churn'] = probs
submission.to_csv('my5_submission.csv', index = False)
